In [7]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

2.5.5


In [8]:
file = open('input.txt', 'r')
query = ""
for ix_row, line in enumerate(file):
    for ix_col, symbol in enumerate(line):
        query_line ="""
CREATE (:Cell {{row:{ix_row}, col:{ix_col}, symbol:'{symbol}'}})\n
            """.format(
                ix_row=ix_row,
                ix_col=ix_col,
                symbol=symbol)
        query += query_line
gds.run_cypher(query)

""


In [9]:
gds.run_cypher("CREATE INDEX cell_row_col IF NOT EXISTS FOR (c:Cell) ON (c.row, c.col)")
gds.run_cypher(MATCH (n {symbol:'\n'}) DETACH DELETE n)

""


In [10]:
queries = """MATCH (c:Cell)
CALL{
    WITH c
    MATCH (x {row:c.row-1, col: c.col})
    MERGE (c)-[:N]->(x)
    } IN TRANSACTIONS OF 1000 ROWS;

MATCH (c:Cell)
CALL{
    WITH c
    MATCH (x {row:c.row-1, col: c.col + 1})
    MERGE (c)-[:NE]->(x)
    } IN TRANSACTIONS OF 1000 ROWS;

MATCH (c:Cell)
CALL{
    WITH c
    MATCH (x {row:c.row, col: c.col + 1})
    MERGE (c)-[:E]->(x)
    } IN TRANSACTIONS OF 1000 ROWS;

MATCH (c:Cell)
CALL{
    WITH c
    MATCH (x {row:c.row + 1, col: c.col + 1})
    MERGE (c)-[:SE]->(x)
    } IN TRANSACTIONS OF 1000 ROWS
""".split(';')
for query in queries:
    gds.run_cypher(query)

In [11]:
gds.run_cypher("MATCH (c:Cell {symbol:'.'}) DETACH DELETE c;")

""


In [12]:
gds.run_cypher("""
MATCH (c:Cell)
WHERE NOT "0123456789" CONTAINS c.symbol
SET c:Symbol
""")

""


In [16]:
gds.run_cypher("""
MATCH (first:Cell&!Symbol WHERE NOT EXISTS {(:Cell&!Symbol)-[:E]->(first)})
    (()-[:E]->(followings:Cell&!Symbol))*
    (last:Cell&!Symbol WHERE NOT EXISTS {(last)-[:E]->(:Cell&!Symbol)})
WHERE any(cell IN [first]+followings WHERE (cell)--(:Symbol))
RETURN sum(toInteger(apoc.text.join([cell IN [first]+followings | cell.symbol],""))) AS part1
""")

,part1
0,537732


In [ ]:
gds.run_cypher("""
MATCH (first:Cell&!Symbol WHERE NOT EXISTS {(:Cell&!Symbol)-[:E]->(first)})
    (()-[:E]->(followings:Cell&!Symbol))*
    (last:Cell&!Symbol WHERE NOT EXISTS {(last)-[:E]->(:Cell&!Symbol)})
WHERE any(cell IN [first]+followings WHERE (cell)--(:Symbol))
WITH [first]+followings AS nodes, toInteger(apoc.text.join([cell IN [first]+followings | cell.symbol],"")) AS part_number
UNWIND nodes AS n
SET n.partNumber = part_number
""")

In [17]:
gds.run_cypher("""
MATCH (s:Symbol {symbol:'*'})--(p:Cell&!Symbol)
WITH s, apoc.coll.toSet(collect(p.partNumber)) AS partNumbers
WHERE size(partNumbers) = 2
RETURN sum(partNumbers[0]*partNumbers[1]) AS part2
""")

,part2
0,84883664
